In [1]:
!which python

/mnt/chihui/anaconda3/bin/python


In [ ]:
# -*- coding: utf-8 -*-
from pattern.text.en import singularize
import json, os, time, parser
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import copy
import logging
from collections import defaultdict
import json, os, time, parser
import _pickle as pickle

#root = os.path.dirname(os.path.abspath(__file__))

def log_conf():

    # set up logging to file - see previous section for more details
    log_file_path = os.path.join('/Users/ch/Documents/medical_entity_dict/modify_entity_dict/all_modify_info.log')
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                        datefmt='%m-%d %H:%M', filename=log_file_path, filemode='w')
    # define a Handler which writes INFO messages or higher to the sys.stderr
    #console = logging.StreamHandler()
    #console.setLevel(logging.INFO)
    # set a format which is simpler for console use
    #formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
    # tell the handler to use this format
    #console.setFormatter(formatter)
    # add the handler to the root logger
    #logging.getLogger('').addHandler(console)
    # logger1 = logging.getLogger('ner_app.area1')

def loads_pickle(map):
    d_c = defaultdict(int)
    d_s = defaultdict(int)
    cate_name=open('/Users/ch/Documents/medical_entity_dict/modify_entity_dict/cate_name.dict','w')
    synoyms_name=open('/Users/ch/Documents/medical_entity_dict/modify_entity_dict/synoyms_name.dict','w')
    if os.path.exists(map):
        fs = open(map, 'rb')
        med_synonyms_name_dict, med_name_cate_dict = pickle.loads(fs.read())  # 使用loads反序列化
#        print(med_synonyms_name_dict, med_name_cate_dict)

    for k,synoyms in  med_name_cate_dict.items():
        string = k + ":" + synoyms + "\n"

        cate_name.write(string)

        d_c[synoyms] +=1
    for k,synoyms in  med_synonyms_name_dict.items():
        string = k + ":" + synoyms + "\n"

        synoyms_name.write(string)
        d_s[synoyms] +=1


    #print (med_name_cate_dict.items())
    #print (d_c.items())
    print (sorted(d_c.items(),key=lambda x:x[1], reverse=True))
    #print (d_s.items())

def gen_dict():
    #with open(os.path.join(root, 'data/medical')) as f:
    fl = open('/Users/ch/Documents/medical_entity_dict/modify_entity_dict/med_name_cate_dict.json','w')
    with open('/Users/ch/Documents/medical_entity_dict/ten_entity_library/medical_entity_dict_v3.0.txt') as f:
        m_e_d = []
        for i in f.readlines():
            try:
                json_text = json.loads(i.strip())
                m_e_d.append(json_text)
            except ValueError:
                logging.debug("json_loads err:{0}".format(i))
    # 存储name->cate,小写（就是category）
    med_name_cate_dict = {}
    # 存储name and synonyms -> name，小写（name:name，synonyms:name，值都name）
    med_synonyms_name_dict = {}

    ####
    print (len(m_e_d))


    #try:
    for item in m_e_d:
        #if item['name']!='3-Hydroxy-3-Methylglutaryl-Coa Lyase Deficiency':
        if item["name"].lower():
            # 如果名字属于名ci，就保留.但如果是形容词，则判断name 是否包含‘，’，‘-’，如果是则保留
            flag_name = pos_tag([item["name"].lower()])[0][-1] in ["NN", "NNS", "NNP", "NNPS"]\
                or (pos_tag([item["name"].lower()])[0][-1] in ["JJ"] and (',' in item['name'] or '-' in item['name']))   #化合物是形容词，保留了特殊符号的
            if flag_name:
#                if (item["name"].lower())[-2:] != "ta":
#                    name = singularize(item["name"].lower())
#                else:
#                    name = item["name"].lower()
#                    
#                med_name_cate_dict[name] = copy.copy(item["category"].lower())
#                med_synonyms_name_dict[name] = copy.copy(name)#建立同义词key与字典key的关联。即将字典name加入同义词中
                med_name_cate_dict[item["name"].lower().strip()] = copy.copy(item["category"].lower())
                med_synonyms_name_dict[item["name"].lower().strip()] = copy.copy(item["name"].lower().strip())#建立同义词key与字典key的关联。即将字典name加入同义词中
            else:
                logging.debug("name pos err-name:{},pos_tag:{}".format(item['name'],pos_tag([item["name"].lower()])))
            # 如果有同义词，并且同义词是名字&&有主键名字，保留
            if item['synonym_list']:
                for synonym in item['synonym_list']:
                    try:
                        flag_synonym = pos_tag([synonym.lower()])[0][-1] in ["NN", "NNS", "NNP", "NNPS"] \
                        or (pos_tag([synonym.lower()])[0][-1] in ["JJ"] and (',' in synonym.lower() or '-' in synonym.lower()))

                        if flag_synonym and flag_name:
                            if (synonym in med_synonyms_name_dict) and (synonym == med_synonyms_name_dict[synonym]):  # 避免将已经存在的{"name":"name"}name实体覆盖掉
                                continue
                            else:
                                med_synonyms_name_dict[synonym.lower().strip()] = copy.copy(item["name"].lower().strip())

                            ###
                            if (item["name"].lower()=='rna, small interfering'):
                                print ('rna, small interfering',item['name'],pos_tag([item["name"].lower()]))
                            ###

                        else :
                            logging.debug("synonmy pos err-name:{},pos_tag:{}".format(item['name'], pos_tag([item["name"].lower()])))
                    except:
                        logging.debug("pos_tag synonym:{0},{1}".format(synonym,item['synonym_list']))

#    med_synonyms_name_dict_black = copy.copy(med_synonyms_name_dict)
#    med_name_cate_dict_black = copy.copy(med_name_cate_dict)
#    ############### 去除忽略的词-实体黑名单#############################
#    with open('/Users/chihui/Documents/medical_entity_dict/black_dict_v2.0.txt') as f:
#        for i in f.readlines():
#            b_key=i.strip().lower()
#            if b_key in med_name_cate_dict.keys():
#                med_name_cate_dict.pop(b_key)
#                logging.debug('cat_pop:{0}'.format(b_key))
#            if b_key in med_synonyms_name_dict.keys():      #同义词中的key去除
#                med_synonyms_name_dict.pop(b_key)
#                logging.debug('syno_pop:{0}'.format(b_key))
#
#            for k,v in med_name_cate_dict_black.items():
#                if len(k) <= 1:
#                    try:
#                        med_name_cate_dict.pop(k)
#                    except:
#                        logging.debug('med_name_cate_dict.pop(k):{}'.format(k))
#                    logging.debug('med_name_cate_pop len<1:{0}'.format(k))
#
#            for k,v in med_synonyms_name_dict_black.items():
#                if b_key == v or len(k) <= 1:    # 去除全部synonyms
#                    if k in med_synonyms_name_dict:
#                        med_synonyms_name_dict.pop(k)
#                        logging.debug('synonyms_value_pop:{0}-{1}'.format(b_key,k))


#    d_for = [med_synonyms_name_dict, med_name_cate_dict]

#    return med_synonyms_name_dict, med_name_cate_dict
    fl.write(json.dumps(med_name_cate_dict))
    fl.close()
    print (len(med_synonyms_name_dict),len(med_name_cate_dict))
    return med_synonyms_name_dict, med_name_cate_dict
    # 序列化
    ##fs.write(pickle.dumps(d_for))

    #fs.close()

    #print (med_name_cate_dict)
    
    
#   实体字典的单复数问题，统计字典中实体通过singularize()转换后单词发生变化的数量；字典中同时包含实体的单数和复数的数量；并统计其中category不相同的情况：
def dict_singular_plural(syn_name_dict, name_cate_dict):
    middle_synonyms_name_dict = copy.copy(syn_name_dict)
    middle_name_cate_dict = copy.copy(name_cate_dict)
    
    singular_name_k_diff = 0
    singular_plural_count = 0
    singular_plural_cate_diff = 0
    for k,v in middle_name_cate_dict.items():
        if (k[-2:] in ["ss","ts","es","hs","ds","gs","fs","ks","ls","ms","bs","cs","qs","rs","ys","ps"]) \
            or (k[-3:] in ["ias","ins","ons","ens","ans","eas","men"]) \
            or (k[-4:] in ["feet"]) \
            or (k[-5:] in ["teeth","women"]) \
            or (k[-8:] in ["children"]):
            singular_name = singularize(k)
            if singular_name != k:
                logging.debug('singular_name_k_diff:{0}#{1}'.format(singular_name, k))
                singular_name_k_diff += 1
            if (singular_name != k) and (singular_name in middle_name_cate_dict):
                logging.debug('dict_singular_plural:{0}#{1}'.format(singular_name, k))
                singular_plural_count += 1
                if middle_name_cate_dict[singular_name] != v:
                    logging.debug('singular_plural_cate_diff:singular_name#{0}#cate#{1}#plural_name#{2} \
                                  #cate#{3}'.format(singular_name, middle_name_cate_dict[singular_name], k, v))
                    singular_plural_cate_diff += 1
    print(str(singular_name_k_diff) + "\n" + str(singular_plural_count) + "\n" + str(singular_plural_cate_diff))


#   对管哥给出的规则进行单复数转换，若单数在字典中存在，不做处理；若不存在，将单数增加到synonyms_name中
def add_singular_to_synonyms(med_synonyms_name_dict, med_name_cate_dict):
    add_synonyms_name_dict = copy.copy(med_synonyms_name_dict)
    add_name_cate_dict = copy.copy(med_name_cate_dict)
    
    for name in add_name_cate_dict:
        if (len(name) > 4) and (name[-5:] != "trans"):
            if (name[-2:] in ["ss","ts","es","hs","ds","gs","fs","ks","ls","ms","bs","cs","qs","rs","ys","ps"]) \
                or (name[-3:] in ["ias","ins","ons","ens","ans","eas","men"]) \
                or (name[-4:] in ["feet"]) \
                or (name[-5:] in ["teeth","women"]) \
                or (name[-8:] in ["children"]):
                singular_name = singularize(name)
                if singular_name not in add_synonyms_name_dict:
                    add_synonyms_name_dict[singular_name.strip()] = name
                    logging.debug('add_singular_to_synonyms:{0}#{1}'.format(singular_name, name))
    print(len(add_synonyms_name_dict),len(add_name_cate_dict))
    return add_synonyms_name_dict, add_name_cate_dict


def dict_label_concat(add_synonyms_name_dict, add_name_cate_dict):
    '''
    合并人工标注的实体，纠正、增加
    遍历人工标注文件，查询sym——name字典，返回name。修改name_cate字典中的category；如果不存在就增加入name_cate,sym-name。
    注意处理大写变小写，单复数问题；（注：其中结尾为"ta"、"ia"的不用进行单复数处理）
    ****** 单复数处理新规则：按早期研发给的规则进行单复数转换******
    category为uncertain的（暂时不做处理）
    '''
    synonyms_name_dict = copy.copy(add_synonyms_name_dict)
    name_category_dict = copy.copy(add_name_cate_dict)
    fl = open('/Users/ch/Documents/medical_entity_dict/modify_entity_dict/name_category_dict.json','w')
    with open('/Users/ch/Documents/medical_entity_dict/modify_entity_dict/correct_add_sent_cate_v1.0') as file:
        
        file.seek(0,0)
        for line in file:
            sent_word = line.split("\t")[0].lower().strip()
#            if sent_word[-2:] != "ta":
#                sent_word = singularize(sent_word)
                
            category = line.split("\t")[1].replace("\n","").strip()
            cate_list = ["drugs","gene","protein","compound","disease","symptom","organism","anatomy","equipment","phenomenon"]
            if category not in cate_list:
                print(sent_word)
                
            if len(sent_word) > 2:
                if sent_word in synonyms_name_dict:
                    name = synonyms_name_dict[sent_word]
                    name_category_dict[name] = category
                    logging.debug('name_cate_modify:{0}#{1}'.format(name, category))
                else:
                    name = sent_word
                    name_category_dict[name] = category
                    synonyms_name_dict[name] = name
                    logging.debug('name_cate_add:{0}#{1}'.format(name, category))
                if sent_word in name_category_dict:
                    name_category_dict[sent_word] = category
                    logging.debug('name_cate_modify:{0}#{1}'.format(sent_word, category))
                    
    fl.write(json.dumps(name_category_dict))
    fl.close()
    print(len(synonyms_name_dict),len(name_category_dict))
    return synonyms_name_dict, name_category_dict


def dict_label_relation_concat(synonyms_name_dict, name_category_dict):
    '''
    将21个有向关系中的所有实体加入字典中，e1、e2 + category
    如果字典中已经存在e1、e2，以字典为准不做修改；如果字典中不存在，就把实体加入到字典的name_cate,sym-name中。
    '''
    syn_name_dict = copy.copy(synonyms_name_dict)
    name_cate_dict = copy.copy(name_category_dict)
    
    fl = open('/Users/ch/Documents/medical_entity_dict/modify_entity_dict/name_cate_dict.json','w')
    file_folder_path = r"/Users/ch/Documents/entity_relation_dict/quchong_relation_dict/"
    file_path_list = []
    for file_path_root,_,file_name_list in os.walk(file_folder_path):
        for file_name in file_name_list:
            if ".dict" in file_name:
                file_path_list.append(os.path.join(file_path_root,file_name))
#    file_name_list = ['compound_protein_treat.dict','disease_disease_coexist.dict','drugs_protein_target.dict',
#                      'drugs_disease_treat.dict','gene_disease_cause.dict','gene_protein_express.dict',
#                      'drugs_disease_treat.dict 2','drugs_gene_target.dict','drugs_protein_target.dict 2','gene_disease_cause.dict 2',
#                      'gene_gene_interact.dict','protein_disease_cause.dict','protein_organism_belong.dict']
    num = 0
    for file_name in file_path_list:
        num = num + 1
        print(num)
#        with open('/Users/ch/Documents/medical_entity_dict/relationship_dict_all/' + file_name) as file:
        with open(file_name,'r') as file:
            file.seek(0,0)
            line = file.readline()
            while(line):            
                line_dict = json.loads(line)
                flag_source = 0
                sources = line_dict["sources"]  #  目前第三批更新的关系中的新实体不加入实体字典中，判断是否来源均为第三批关系字典
                if ("-1" in sources) or ("-2" in sources) or ("-3" in sources) or ("-4" in sources) or ("-5" in sources) or \
                    ("-6" in sources) or ("-7" in sources) or ("-8" in sources) or ("-9" in sources):  # ("-1"--"-9"为前两批的关系字典)
                    flag_source = 1
                if flag_source == 1:
                    e1 = line_dict["e1"].lower().strip()
                    e1_category = os.path.basename(file_name).split("_")[0]  #实体e1的category
                    if e1:
                        entity1 = pos_tag([e1])[0][-1] in ["NN", "NNS", "NNP", "NNPS"]\
                            or (pos_tag([e1])[0][-1] in ["JJ"] and (',' in e1 or '-' in e1))   #化合物是形容词，保留了特殊符号的
                        if entity1 and len(e1) > 3:
                            if e1 not in syn_name_dict:
                                name_cate_dict[e1] = e1_category
                                syn_name_dict[e1] = e1
                                logging.debug('name_cate_relation_add:{0}#{1}'.format(e1, e1_category))
                        else:
                            logging.debug('relation_dict_entity_length(0_3)pop:{0}'.format(e1, e1_category))  # 将关系字典中实体长度小于等于3的实体筛掉
    #                    if e1 == "to" or e1 == "be":
    #                        print(e1, e1_category)
    
                    e2 = line_dict["e2"].lower().strip()
                    e2_category = os.path.basename(file_name).split("_")[1]  #实体e2的category
                    if e2:
                        entity2 = pos_tag([e2])[0][-1] in ["NN", "NNS", "NNP", "NNPS"]\
                            or (pos_tag([e2])[0][-1] in ["JJ"] and (',' in e2 or '-' in e2))   #化合物是形容词，保留了特殊符号的
                        if entity2 and len(e2) > 3:
                            if e2 not in syn_name_dict:
                                name_cate_dict[e2] = e2_category
                                syn_name_dict[e2] = e2
                                logging.debug('name_cate_relation_add:{0}#{1}'.format(e2, e2_category))
                        else:
                            logging.debug('relation_dict_entity_length(0_3)pop:{0}'.format(e2, e2_category))  # 将关系字典中实体长度小于等于3的实体筛掉
    #                    if e2 == "to" or e2 == "be":
    #                        print(e2, e2_category)
                    
    #                if e2[-2:] != "ta":
    #                    e2 = singularize(e2)
                line = file.readline()

    print(len(syn_name_dict),len(name_cate_dict))
    fl.write(json.dumps(name_cate_dict))
    fl.close()
    return syn_name_dict, name_cate_dict
    ############### 去除忽略的词-实体黑名单#############################    
def del_black_dict(syn_name_dict, name_cate_dict):
#   删除黑名单中的实体
    med_synonyms_name_dict_black = copy.copy(syn_name_dict)
    med_name_cate_dict_black = copy.copy(name_cate_dict)
    fl = open('/Users/ch/Documents/medical_entity_dict/modify_entity_dict/name_cate_dict_final.json','w')
    fs = open('/Users/ch/Documents/medical_entity_dict/modify_entity_dict/medical_entity_dict_v4.3.pkl','wb')
    fs_json = open('/Users/ch/Documents/medical_entity_dict/modify_entity_dict/medical_entity_dict_v4.3.json','w')
    with open('/Users/ch/Documents/medical_entity_dict/black_dict_v3.0.txt') as f:        
        for i in f.readlines():
            b_key = i.strip().lower()   #做大写变小写处理
            if b_key in name_cate_dict.keys():
                name_cate_dict.pop(b_key)
                logging.debug('cat_pop:{0}'.format(b_key))
            if b_key in syn_name_dict.keys():      #同义词中的key去除
                syn_name_dict.pop(b_key)
                logging.debug('syno_pop:{0}'.format(b_key))

            for k,v in med_synonyms_name_dict_black.items():
                if b_key == v or len(k) <= 2:    # 如果name在黑名单里，或者长度小于等于1，去除name对应的全部synonyms,
#                if b_key == v or len(k) <= 2 or len(v) <= 2:
                    if k in syn_name_dict:
                        syn_name_dict.pop(k)
                        logging.debug('synonyms_value_pop:{0}#{1}'.format(b_key,k))
#    如果 synonyms长度大于2，name长度小于等于2，那么就保留 synonyms 删除 name。                        
#                elif len(k) > 2 and len(v) <= 2:    
#                    syn_name_dict[k] = k
#                    name_cate_dict[k] = name_cate_dict[v]
#                    logging.debug('synonyms_retain_name_pop:{0}#{1}'.format(k,v))
                    
        for k,v in med_synonyms_name_dict_black.items():
            if len(k) > 2 and len(v) <= 2:
                if k in syn_name_dict:
                    syn_name_dict[k] = k
                    name_cate_dict[k] = name_cate_dict[v]
                    logging.debug('synonyms_retain_name_pop:{0}#{1}'.format(k,v))
                    
        for k,v in med_name_cate_dict_black.items():
            if len(k) <= 2:
                try:
                    name_cate_dict.pop(k)
                except:
                    logging.debug('name_cate_dict.pop(k):{0}'.format(k))
                logging.debug('name_cate_pop len<=2:{0}'.format(k))
#    将 adora2a 的同义词 adenosine a2a receptor、adenosine receptor a2a 归一到 adora2a 上。
        syn_name_dict["adenosine a2a receptor"] = "adora2a"
        syn_name_dict["adenosine receptor a2a"] = "adora2a"
#        name_cate_dict.pop("adenosine a2a receptor")
#        name_cate_dict.pop("adenosine receptor a2a")
        print(len(syn_name_dict),len(name_cate_dict))
    fl.write(json.dumps(name_cate_dict))
    for k,v in name_cate_dict.items():
        if (k=='isoleucine'):
            print ('debug::::::::',k,v)
    fl.close()
    # 序列化
    dict_concat = [syn_name_dict, name_cate_dict]
    pickle.dump(dict_concat, fs)
    json.dump(dict_concat,fs_json,indent=4)
    
    fs.close()
    return(syn_name_dict, name_cate_dict)


#    print (med_name_cate_dict)


log_conf()

med_synonyms_name_dict, med_name_cate_dict = gen_dict()

add_synonyms_name_dict, add_name_cate_dict = add_singular_to_synonyms(med_synonyms_name_dict, med_name_cate_dict)
synonyms_name_dict, name_category_dict = dict_label_concat(add_synonyms_name_dict, add_name_cate_dict)
syn_name_dict, name_cate_dict = dict_label_relation_concat(synonyms_name_dict, name_category_dict)
syn_name_dict, name_cate_dict = del_black_dict(syn_name_dict, name_cate_dict)


#dict_singular_plural(med_synonyms_name_dict, med_name_cate_dict)
#loads_pickle('/Users/ch/Documents/medical_entity_dict/relationship_dict/dict.pkl')
#dict_singular_plural(syn_name_dict, name_cate_dict)


In [1]:
import pattern

ImportError: bad magic number in 'pattern': b'\x03\xf3\r\n'

In [3]:
import pattern.en

ModuleNotFoundError: No module named 'pattern.en'

In [6]:
import sys

In [7]:
print (sys.executable)

/mnt/chihui/anaconda3/bin/python


In [8]:
MODULE = '/mnt/chihui/anaconda3/lib/python3.7/site-packages/pattern'
import sys
if MODULE not in sys.path: sys.path.append(MODULE)
from pattern.en import parsetree

ModuleNotFoundError: No module named 'pattern.en'

In [16]:
import pattern

In [2]:
dir(pattern)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [3]:
!which python

/mnt/chihui/anaconda3/bin/python
